In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from os import read
dfs = spark \
.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.csv("/content/drive/MyDrive/Colab/DataScience/Spark/titanic.csv")

dfs.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
dfs.count()

891

In [ ]:
dfs.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
dfs.select("Name", "Age").show(3)

+--------------------+----+
|                Name| Age|
+--------------------+----+
|Braund, Mr. Owen ...|22.0|
|Cumings, Mrs. Joh...|38.0|
|Heikkinen, Miss. ...|26.0|
+--------------------+----+
only showing top 3 rows



In [ ]:
dfs.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [ ]:
dfs.describe('age', 'fare').show()

+-------+------------------+-----------------+
|summary|               age|             fare|
+-------+------------------+-----------------+
|  count|               714|              891|
|   mean| 29.69911764705882| 32.2042079685746|
| stddev|14.526497332334035|49.69342859718089|
|    min|              0.42|              0.0|
|    max|              80.0|         512.3292|
+-------+------------------+-----------------+



In [ ]:
dfs.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [ ]:
dfs.groupBy('Survived','Sex').count().show()

+--------+------+-----+
|Survived|   Sex|count|
+--------+------+-----+
|       0|female|   81|
|       1|  male|  109|
|       1|female|  233|
|       0|  male|  468|
+--------+------+-----+



In [ ]:
dfs.groupBy('Survived','Sex').avg().show()

+--------+------+------------------+-------------+------------------+------------------+-------------------+-------------------+------------------+
|Survived|   Sex|  avg(PassengerId)|avg(Survived)|       avg(Pclass)|          avg(Age)|         avg(SibSp)|         avg(Parch)|         avg(Fare)|
+--------+------+------------------+-------------+------------------+------------------+-------------------+-------------------+------------------+
|       0|female|434.85185185185185|          0.0|2.8518518518518516|         25.046875| 1.2098765432098766|  1.037037037037037| 23.02438518518519|
|       1|  male|475.72477064220186|          1.0| 2.018348623853211|27.276021505376345| 0.3853211009174312| 0.3577981651376147| 40.82148440366974|
|       1|female|429.69957081545067|          1.0|1.9184549356223175| 28.84771573604061| 0.5150214592274678| 0.5150214592274678| 51.93857339055791|
|       0|  male|449.12179487179486|          0.0|2.4764957264957266|31.618055555555557|0.44017094017094016|0.20

In [ ]:
dfs.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [ ]:
dfs = dfs.na.fill({'Embarked' : 'S'})

In [ ]:
dfs.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+



In [ ]:
dfs.drop('Cabin')

dfs.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = false)



In [ ]:
dfs.filter(dfs.Age > 70).show()

+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-----+--------+
|         97|       0|     1|Goldschmidt, Mr. ...|male|71.0|    0|    0|PC 17754|34.6542|   A5|       C|
|        117|       0|     3|Connors, Mr. Patrick|male|70.5|    0|    0|  370369|   7.75| null|       Q|
|        494|       0|     1|Artagaveytia, Mr....|male|71.0|    0|    0|PC 17609|49.5042| null|       C|
|        631|       1|     1|Barkworth, Mr. Al...|male|80.0|    0|    0|   27042|   30.0|  A23|       S|
|        852|       0|     3| Svensson, Mr. Johan|male|74.0|    0|    0|  347060|  7.775| null|       S|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-----+--------+

